# Анализ тональности с BERT

## Описание исследования

Интернет-магазин «Викишоп» запускает новый сервис. Пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

## Цель исследования

- Обучить модель классифицировать комментарии как позитивные или негативные. Построить модель со значением метрики качества F1 не меньше 0.75.

## Ход исследования

1. Загрузка и подготовка данных
2. Обучение моделей
2. Выводы

## Описание данных

- 'text' - текст комментарий
- 'toxic' - целевой признак, является ли комментарий токсичным

<a id='section_id'></a>
## Содержание 

[Шаг 1. Загрузка данных](#section_id1)

[Шаг 2. Предобработка и исследовательский анализ](#section_id2)

[Шаг 3. Подготовка данных](#section_id3)

[Шаг 4. Обучение моделей](#section_id4)

[Шаг 5. Проверка на тестовой выборке](#section_id5)

[Шаг 6. Вывод](#section_id6)

In [1]:
# импорт библиотек

# работа с данными
import pandas as pd
import numpy as np

# подготовка данных
from sklearn.model_selection import train_test_split
import transformers
import torch

# модели машинного обучения
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# пайплайны
from sklearn.pipeline import Pipeline

# инструменты поиска
from sklearn.model_selection import GridSearchCV

# инструменты управления ресурсами
import joblib
import warnings
from tqdm import notebook

# метрика для оценки прогноза
from sklearn.metrics import f1_score

In [2]:
# константы
TEST_SIZE = 0.25 
RANDOM_STATE = 42
BATCH_SIZE = 2

# настройки
warnings.filterwarnings('ignore')

<a id='section_id1'></a>
## Шаг 1. Загрузка данных
[к содержанию](#section_id)

In [3]:
# загрузка данных
df = pd.read_csv('..\\data\\toxic_comments.csv')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<a id='section_id2'></a>
## Шаг 2. Предобработка и исследовательский анализ
[к содержанию](#section_id)

In [4]:
# удаление столбца без названия и сокращение датасета
df = df[['text', 'toxic']].sample(1000)

In [5]:
# функция для обзора данных
def preview(dataset):
    '''Функция принимает на вход набор данных и выводит основную информацию о нем.'''
    display(dataset.head())
    dataset.info()
    display(dataset.describe(include='all').T)

In [6]:
# обзор данных
preview(df)

,text,toxic
69682,"""\n Okay, it's clear English is your second la...",0
120861,"do you mean by seems to? More like does, dumba...",1
149311,What did I say that was incorrect about the Sa...,0
138485,"Just an idea now, it will come soon.",0
142501,", 12 Apr 2005 (UTC)\n\nIf the claim is that th...",0


<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 69682 to 138304
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   toxic   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 23.4+ KB


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
text,1000,1000,I am working from sources. I'm not sure about ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
toxic,1000.0,NaN,NaN,NaN,0.122,0.32745,0.0,0.0,0.0,0.0,1.0


In [7]:
# определение баланса классов
df['toxic'].value_counts()

toxic
0    878
1    122
Name: count, dtype: int64

Выводы о данных:
- пропусков нет
- повторяющихся комментариев нет
- типы данных приведены верно
- баланс классов смещен в сторону нетоксичных комментариев

<a id='section_id3'></a>
## Шаг 3. Подготовка данных
[к содержанию](#section_id)

In [8]:
# инициализация токенизатора
tokenizer = transformers.BertTokenizer(
    vocab_file='..\\data\\bert\\vocab.txt')

# запуск токенизации
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

# определение максимальной длины вектора
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

# дополнение векторов меньшей длины нулями
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

# срез по размерности токенов
padded = padded[:, :512]

# создание маски внимания
attention_mask = np.where(padded != 0, 1, 0)

In [9]:
attention_mask.shape

(1000, 512)

In [10]:
# инициализация конфигурации
config = transformers.BertConfig.from_json_file(
    '..\\data\\bert\\config.json')

# инициализация модели
model = transformers.BertModel.from_pretrained(
    '..\\data\\bert\\pytorch_model.bin', config=config)

In [11]:
# создание эмбедингов
embeddings = []

with joblib.parallel_backend("threading"):
    for i in notebook.tqdm(range(padded.shape[0] // BATCH_SIZE)):
            batch = torch.LongTensor(padded[BATCH_SIZE*i:BATCH_SIZE*(i+1)]) 
            attention_mask_batch = torch.LongTensor(attention_mask[BATCH_SIZE*i:BATCH_SIZE*(i+1)])
            
            with torch.no_grad():
                batch_embeddings = model(batch, attention_mask=attention_mask_batch)
            
            embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/500 [00:00<?, ?it/s]

In [12]:
# сборка признаков
features = np.concatenate(embeddings)

# разбиение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    features,
    df['toxic'],
    test_size = TEST_SIZE,
    stratify=df['toxic'],
    random_state = 42)

<a id='section_id4'></a>
## Шаг 4. Обучение моделей
[к содержанию](#section_id)

In [13]:
# пайплайн обучения
pipe_final = Pipeline([
    ('models', LogisticRegression(random_state=RANDOM_STATE))
])

In [14]:
# задание параметров для пайплайна
param_grid = [
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE,
            solver='liblinear',
            penalty='l2'
        )],
        'models__C': [5, 10, 15],
    },
    
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_features': range(6, 8),
        'models__max_depth': range(8, 10)
    },
    
    # словарь для модели KNeighborsClassifier() 
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': [5, 25]   
    }
]

In [15]:
# инициализация подбора параметров
grid_search = GridSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

In [16]:
%%time
# запуск подбора параметров
with joblib.parallel_backend("threading"):
        grid_search.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', grid_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', grid_search.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Лучшая модель и её параметры:

 Pipeline(steps=[('models',
                 LogisticRegression(C=5, random_state=42, solver='liblinear'))])
Метрика лучшей модели на тренировочной выборке: 0.5463741177867985
CPU times: total: 14.5 s
Wall time: 3.33 s


In [17]:
# получение результатов лучших моделей
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values(by='rank_test_score', inplace=True)
results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_models,param_models__C,param_models__max_depth,param_models__max_features,param_models__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.840721,0.117201,0.014634,0.007756,"LogisticRegression(random_state=42, solver='li...",5.0,NaN,NaN,NaN,"{'models': LogisticRegression(random_state=42,...",0.620690,0.611111,0.322581,0.571429,0.606061,0.546374,0.113126,1
1,0.799845,0.053563,0.008203,0.003475,"LogisticRegression(random_state=42, solver='li...",10.0,NaN,NaN,NaN,"{'models': LogisticRegression(random_state=42,...",0.600000,0.578947,0.285714,0.571429,0.606061,0.528430,0.122033,2
2,0.725678,0.036819,0.008806,0.004705,"LogisticRegression(random_state=42, solver='li...",15.0,NaN,NaN,NaN,"{'models': LogisticRegression(random_state=42,...",0.600000,0.564103,0.277778,0.571429,0.588235,0.520309,0.121916,3
3,0.020434,0.010835,0.004444,0.000552,DecisionTreeClassifier(random_state=42),NaN,8.0,6.0,NaN,{'models': DecisionTreeClassifier(random_state...,0.285714,0.222222,0.352941,0.250000,0.187500,0.259676,0.056714,4
6,0.012887,0.001389,0.004839,0.000709,DecisionTreeClassifier(random_state=42),NaN,9.0,7.0,NaN,{'models': DecisionTreeClassifier(random_state...,0.227273,0.235294,0.216216,0.307692,0.294118,0.256119,0.037314,5
5,0.011990,0.001650,0.004806,0.000659,DecisionTreeClassifier(random_state=42),NaN,9.0,6.0,NaN,{'models': DecisionTreeClassifier(random_state...,0.242424,0.333333,0.263158,0.200000,0.176471,0.243077,0.054483,6
4,0.012439,0.001544,0.005833,0.001963,DecisionTreeClassifier(random_state=42),NaN,8.0,7.0,NaN,{'models': DecisionTreeClassifier(random_state...,0.187500,0.279070,0.068966,0.363636,0.142857,0.208406,0.103215,7
7,0.010210,0.003553,1.085882,0.056478,KNeighborsClassifier(),NaN,NaN,NaN,5.0,"{'models': KNeighborsClassifier(), 'models__n_...",0.181818,0.320000,0.153846,0.000000,0.250000,0.181133,0.107313,8
8,0.009018,0.003193,0.611344,0.465054,KNeighborsClassifier(),NaN,NaN,NaN,25.0,"{'models': KNeighborsClassifier(), 'models__n_...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9


<a id='section_id5'></a>
## Шаг 5. Проверка на тестовой выборке
[к содержанию](#section_id)

In [18]:
# проверка лучшей модели на тестовой выборке
pred = grid_search.best_estimator_.predict(X_test)

print("f1 тестовой выборки:", f1_score(y_test, pred))

f1 тестовой выборки: 0.5


<a id='section_id6'></a>
## Шаг 6. Вывод
[к содержанию](#section_id)

Решена задача по классификации токсичных комментариев. При создании тренировочной и тестовой выборки выявлено, что токсичных коментариев значительно меньше, поэтому применена стратификация, чтобы равномерно распределить комментарии каждого класса по выборкам. Лемматизация произведена с помощью pymystem3 внутри функции, фильтровались специальные символы и стоп-слова. Для получения наборов признаков для обучения моделей использовался CountVectorizer, так как он показал лучшие результаты в сравнении с TfIdfVectorizer. Обучение производилось в пайплайне: исследовались модели LogisticRegression, DecisionTreeClassifier, KNeighborsClassifier. Лучшей моделью стала LogisticRegression с параметром регуляризации C=15. Неплохие результаты показала модель KNeighborsClassifier с параметрами n_neighbors=5. На тестовой выборке лучшая модель показала значение метрики f1_score равным 0.78